In [ ]:
!pip install opencv-contrib-python

In [ ]:
! pip3 install imutils

In [ ]:
! pip3 install easydict

In [ ]:
from imutils.video import VideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import sys
from config_reader import config_reader
from processing import extract_parts, draw
from model.cmu_model import get_testing_model

In [ ]:
from easydict import EasyDict as edict

In [ ]:
args = edict({
    
    "--prototxt": 'model/keras/model.h5',
    "--model": 'training/train_pose',
    "--confidence": 0.2,
    "--process_speed": 3,
    "--end": 80,
    '--frame_ratio':518400    
    
})

process_speed=args
ending_frame=args





In [ ]:
print("loading model...")
from model.cmu_model import get_testing_model
model = get_testing_model()
model.load_weights('model/keras/model.h5')



In [ ]:
#setting path for video output

# Video output
video = 'sample1_test5.mp4'
video_path = 'videos/outputs'
video_file = video_path + video

# Output location
output_path = 'videos/outputs/'
output_format = '.mp4'

#timestamp
currentDT = time.localtime()
start_datetime = time.strftime("-%m-%d-%H-%M-%S", currentDT)

#video path for output
video_output = output_path + video + str(start_datetime) + output_format


In [ ]:
#input video

# loading video
#cam has the input video
cam = cv2.VideoCapture('videos/sample2.mp4')

a=cam.get(cv2.CAP_PROP_FRAME_WIDTH)
print(a)
b=cam.get(cv2.CAP_PROP_FRAME_HEIGHT)
print(b)
input_fps = cam.get(cv2.CAP_PROP_FPS)
print(input_fps)

ret,frame=cam.read()
print(ret)

print(cam.read())




#reading the video file
ret_val,orig_video = cam.read()
#print(ret_val)
video_length = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))
print(video_length)

#ending fram is the length of the video
#which will be the last frame
ending_frame = 80



output_fps = 29.97/518400

#video writer
#input frames in the video, we use videowriter from opencv
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(video_output, fourcc,  output_fps, (orig_video.shape[1], orig_video.shape[0]))
print(out)


In [ ]:
# load config
params, model_params = config_reader()
scale_search = [1, .5, 1.5, 2]  # [.5, 1, 1.5, 2]
scale_search = scale_search[0:4]

params['scale_search'] = scale_search



In [ ]:

i = 0  # default is 0
while(cam.isOpened()) and ret_val is True and i < ending_frame:
        if i % 1 == 0:

            #applying colour to the input video file 
            input_video_colour = cv2.cvtColor(orig_video, cv2.COLOR_RGB2BGR)
           # print(input_video_colour)

            tic = time.time()

            # detecting the body parts in the image
            all_peaks, subset, candidate = extract_parts(input_video_colour, params, model, model_params)
            canvas = draw(orig_video, all_peaks, subset, candidate)

            print('Processing frame: ', i)
            toc = time.time()
            print('processing time is %.5f' % (toc - tic))

            out.write(canvas)
            #print(out)
            
           
            ret_val, orig_video = cam.read()
            i += 1